# Summarize Data

The `arcgis.geoanalytics.summarize_data` submodule contains functions that calculate total counts, lengths, areas, and basic descriptive statistics of features and their attributes within areas or near other features.

<h2>Table of contents</h2>
    
- [Aggregate Points](#Aggregate_Points)
- [Build Multi-Variable Grid](#Build_Multi-Variable_Grid)
- [Describe Dataset](#Describe_Dataset)
- [Reconstruct Tracks](#Reconstruct_Tracks)
- [Summarize Attributes](#Summarize_Attributes)
- [Summarize Center and Dispersion](#Summarize_Center_and_Dispersion)
- [Summarize Within](#Summarize_Within)


This toolset uses distributed processing to complete analytics on your GeoAnalytics Server.

<table>
  <tr>
    <th><center>Tool</center></th>
    <th><center>Description</center></th>    
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalyticstool-aggregatepoints.htm"><p align="left">aggregate_points</p></a></td>
      <td><p align="left">Aggregates points into polygon features or bins. A polygon is returned with a count of points as well as optional statistics at all locations where points exist.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-build-multi-variable-grid.htm"><p align="left">build_multivariable_grid</p></a></td>
      <td><p aligh="left">Generates a grid of square or hexagonal bins and calculates variables for each bin based on the proximity of one or more input layers.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-describe-dataset.htm"><p align="left">describe_dataset</p></a></td>
      <td><p align="left">Summarizes features into calculated field statistics, sample features, and extent boundaries.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-join-features.htm"><p align="left">join_features</p></a></td>
      <td><p align="left">Joins attributes from one layer to another based on spatial, temporal, or attribute relationships, or a combination of those relationships.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-reconstruct-tracks.htm"><p align="left">reconstruct_tracks</p></a></td>
      <td><p align="left">Creates line or polygon tracks from time-enabled input data.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-summarize-attributes.htm"><p align="left">summarize_attributes</p></a></td>
      <td><p align="left">Calculates summary statistics for fields in a feature class.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalyticstool-aggregatepoints.htm"><p align="left">summarize_center_and_dispersion</p></a></td>
      <td><p align="left">Finds central features and directional distributions and calculates mean and median locations from the input.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-summarize-within.htm"><p align="left">summarize_within</p></a></td>
      <td><p align="left">Overlays a polygon layer with another layer to summarize the number of points, length of the lines, or area of the polygons within each polygon and calculates attribute field statistics about those features within the polygons.</p></td>
  </tr>   
 

**Note**: The purpose of the notebook is to show examples of the different tools that can be run on an example dataset.

<b>Connect to your gis

In [10]:
# connect to Enterprise GIS
from arcgis.gis import GIS
import arcgis.geoanalytics

portal_gis = GIS("https://pythonapi.playground.esri.com/portal", "arcgis_python", "amazing_arcgis_123")

<b>Search items registered with your server

In [4]:
search_result = portal_gis.content.search("bigDataFileShares_all_hurricanes", item_type = "big data file share")[0]
search_result

<Item title:"bigDataFileShares_all_hurricanes" type:Big Data File Share owner:api_data_owner>

In [5]:
years_50 = search_result.layers[0]

In [2]:
search_result = portal_gis.content.search("bigDataFileShares_all_hurricanes", item_type = "big data file share")[0]
search_result

<Item title:"bigDataFileShares_all_hurricanes" type:Big Data File Share owner:api_data_owner>

In [3]:
search_result.layers

[<Layer url:"https://pythonapi.playground.esri.com/ga/rest/services/DataStoreCatalogs/bigDataFileShares_all_hurricanes/BigDataCatalogServer/hurricanes">]

In [5]:
search_result.layers

[<Layer url:"https://pythonapi.playground.esri.com/ga/rest/services/DataStoreCatalogs/bigDataFileShares_GA_Data/BigDataCatalogServer/air_quality">,
 <Layer url:"https://pythonapi.playground.esri.com/ga/rest/services/DataStoreCatalogs/bigDataFileShares_GA_Data/BigDataCatalogServer/crime">,
 <Layer url:"https://pythonapi.playground.esri.com/ga/rest/services/DataStoreCatalogs/bigDataFileShares_GA_Data/BigDataCatalogServer/calls">,
 <Layer url:"https://pythonapi.playground.esri.com/ga/rest/services/DataStoreCatalogs/bigDataFileShares_GA_Data/BigDataCatalogServer/analyze_new_york_city_taxi_data">]

In [12]:
search_result = portal_gis.content.search("bigDataFileShares_ServiceCallsOrleans", item_type = "big data file share")[0]
search_result

<Item title:"bigDataFileShares_ServiceCallsOrleans" type:Big Data File Share owner:portaladmin>

In [13]:
calls = search_result.layers[0]

In [7]:
search_result = portal_gis.content.search("bigDataFileShares_ServiceCallsOrleans", item_type = "big data file share")[0]
search_result

<Item title:"bigDataFileShares_ServiceCallsOrleans" type:Big Data File Share owner:portaladmin>

In [8]:
calls = search_result.layers[0]

In [6]:
calls = search_result[0].layers[2]
hurricanes = search_result.lyers[1]

## Aggregate Points

<center><img src="../../static/img/guide_img/ga/aggregate_points.png" height="300" width="300"></center>

This [`aggregate_points`](https://developers.arcgis.com/rest/services-reference/enterprise/aggregate-points.htm) tool works with a layer of point features and a layer of areas. The layer of areas can be an input polygon layer or it can be square or hexagonal bins calculated when the task is run. The tool first determines which points fall within each specified area. After determining this point-in-area spatial relationship, statistics about all points in the area are calculated and assigned to the area. The most basic statistic is the count of the number of points within the area, but you can get other statistics as well.

This tool can also work on data that is time-enabled. If time is enabled on the input points, then the time slicing options are available. Time slicing allows you to calculate the point-in area relationship while looking at a specific slice in time. For example, you could look at hourly intervals, which would result in outputs for each hour.

For an example with time, suppose you had point features of every transaction made at a coffee shop location and no area layer. The data has been recorded over a year, and each transaction has a location and a time stamp. Assuming each transaction has a TOTAL_SALES attribute, you can get the sum of all TOTAL SALES within the space and time of interest. If these transactions are for a single city, we could generate areas that are one kilometer grids, and look at weekly time slices to summarize the transactions in both time and space.

In [5]:
from arcgis.geoanalytics.summarize_data import aggregate_points

The example below shows how you could aggregate your data based on the area of hexagonal bins. 

In [8]:
output1 = aggregate_points(point_layer=calls, #the input point layer to be aggregated
                          bin_type='Hexagon', #type of bins to be created
                          bin_size=1, # size of bin 
                          bin_size_unit='Meters', # unit of bin
                          output_name='aggregate output') # output name for aggregated result

{"messageCode":"BD_101068","message":"Bin generation and analysis requires a projected coordinate system and a default projection of World Cylindrical Equal Area has been applied."}
{"messageCode":"BD_101054","message":"Some records have either missing or invalid geometries."}
{"messageCode":"BD_101088","message":"Some result features were clipped to the valid extent of the resulting spatial reference."}


You could also choose to aggregate your data within a polygon. You will need to provide polygon_layer parameter in order to do so.

In [ ]:
output2 = aggregate_points(point_layer=calls, 
                           polygon_layer=
                           time_step_interval=None,
                           time_step_interval_unit=None,
                           time_step_repeat_interval=None,
                           time_step_repeat_interval_unit=None,
                           time_step_reference=None)
output2

You might have noticed that output2 is a FeatureCollection while output1 was a feature layer. If output_name parametr is passed, it saves the result as a feature layer item, otherwise it returns a FeatureCollection. summary_fields parameter is an important one which controls what statistical summary types will be calculated for all points within each polygon or bin. Note that, count of all points withina polygon is always returned.

In [ ]:
output3 = aggregate_points(point_layer=calls, 
                           polygon_layer=,
                           summary_fields={}, 
                           output_name=)

If your input points are time-enabled and represent an instant in time, you could also aggregate data based on time period.

In [ ]:
output4 = aggregate_points(point_layer=calls, 
                           polygon_layer=
                           time_step_interval=None,
                           time_step_interval_unit=None,
                           time_step_repeat_interval=None,
                           time_step_repeat_interval_unit=None,
                           time_step_reference=None,
                           summary_fields=None)

To learn more about aggregate_points tool, please read [api-reference](https://developers.arcgis.com/python/api-reference/1.8.4/arcgis.geoanalytics.summarize_data.html#aggregate-points) for help.

## Build Multi-Variable Grid

<center><img src="../../static/img/guide_img/ga/grid.png" height="300" width="300"></center>

The `build_multivariable_grid` task works with one or more layers of point, line, or polygon features. The task generates a grid of square or hexagonal bins and compiles information about each input layer into each bin. For each input layer, this information can include the following variables:

- Distance to Nearest - The distance from each bin to the nearest feature.

- Attribute of Nearest - An attribute value of the feature nearest to each bin.

- Attribute Summary of Related - A statistical summary of all features within search_distance of each bin.

Only variables you specify in variable_calculations will be included in the result layer. These variables can help you understand the proximity of your data throughout the extent of your analysis. The results can help you answer questions such as the following:

Given multiple layers of public transportation infrastructure, what part of the city is least accessible by public transportation?

Given layers of lakes and rivers, what is the name of the water body closest to each location in the U.S.?

Given a layer of household income, where in the U.S. is the variation of income in the surrounding 50 miles the greatest?

The result of `build_multivariable_grid` can also be used in prediction and classification workflows. The task allows you to calculate and compile information from many different data sources into a single, spatially continuous layer in one step. This layer can then be used with the Enrich From Multi-Variable Grid task to quickly enrich point features with the variables you have calculated, reducing the amount of effort required to build prediction and classification models from point data.m

In [11]:
from arcgis.geoanalytics.summarize_data import build_multivariable_grid

In [15]:
var_calc = [{"layer":0,"variables":[{"type":"AttributeOfNearest","outFieldName":"test","attributeField":"Location","searchDistance":6,"searchDistanceUnit":"Miles"}]}]

This example creates a multivariable grid by summarizing information the attribute of nearest location.

In [16]:
##usage example
output = build_multivariable_grid(input_layers=[calls], 
                                  variable_calculations=var_calc, 
                                  bin_size=5, 
                                  bin_unit='Miles', 
                                  bin_type='Square', 
                                  output_name='build_multivariable_grid')
output

<Item title:"build_multivariable_grid" type:Feature Layer Collection owner:arcgis_python>

## Describe dataset

The [`describe_dataset`](https://developers.arcgis.com/rest/services-reference/enterprise/describe-dataset.htm) task provides an overview of your big data. The tool outputs a feature layer representing a sample of your input features or a single polygon feature layer that represents the extent of your input features. You can choose to output one, both, or none.

Optionally, the tool can also output a feature layer representing a sample of your input features or a single polygon feature layer that represents the extent of your input features. You can choose to output one, both, or none.

For example, imagine you are tasked with completing an analysis workflow on a large volume of data. You want to try the workflow, but it could take hours or days with your full dataset. Instead of using time and resources running the full analysis, first create a sample layer to efficiently test your workflow before running it on the full dataset.

<center><img src="../../static/img/guide_img/ga/describe_dataset.png" height="300" width="300"></center>

In [10]:
from arcgis.geoanalytics.summarize_data import describe_dataset
from datetime import datetime as dt

In [11]:
description = describe_dataset(input_layer=calls,
                               extent_output=True,
                               sample_size=1000,
                               output_name="Description of service calls" + str(dt.now().microsecond),
                               return_tuple=True)

In [15]:
description.output

<FeatureLayer url:"https://pythonapi.playground.esri.com/server/rest/services/Hosted/Description_of_service_calls645206/FeatureServer/0">

In [16]:
description.output_json

{'datasetName': 'yearly_calls',
 'datasetSource': 'Big Data File Share - ServiceCallsOrleans',
 'recordCount': 510153,
 'geometry': {'geometryType': 'Point',
  'sref': {'wkid': 102682, 'latestWkid': 3452},
  'countNonEmpty': 510153,
  'countEmpty': 0,
  'spatialExtent': {'xmin': 3659898,
   'ymin': 501901,
   'xmax': 37369000,
   'ymax': 3513814}},
 'time': {'timeType': 'Instant',
  'countNonEmpty': 510153,
  'countEmpty': 0,
  'temporalExtent': {'start': '2011-01-01 00:00:02.000',
   'end': '2011-12-31 23:57:11.000'}}}

## Reconstruct tracks

The Reconstruct Tracks tool connects time-sequential points to tracks and summarizes features within the track. Tracks are identified by one or more track fields. The resulting layer displays the track as a line or an area, the count of features within a track that have been summarized, and any additional statistics that have been specified.

<center><img src="../../static/img/guide_img/ga/reconstruct_tracks.png" height="300" width="300"></center>

The [`reconstruct_tracks`](https://developers.arcgis.com/rest/services-reference/enterprise/reconstruct-tracks.htm) task works with a time-enabled layer of either point or polygon features that represents an instant in time. It first determines which features belong to a track using an identifier. Using the time at each location, the tracks are ordered sequentially and transformed into a line or polygon representing the path of movement over time. Optionally, the input can be buffered by a field, which will create a polygon at each location. These buffered points, or polygons if the inputs are polygons, are then joined sequentially to create a track as a polygon where the width is representative of the attribute of interest. Resulting tracks have start and end times that represent the time at the first and last feature in a given track. When the tracks are created, statistics about the input features are calculated and assigned to the output track. The most basic statistic is the count of points within the area, but other statistics can be calculated as well. Features in time-enabled layers can be represented in one of two ways:

- Instant - A single moment in time

- Interval - A start and end time

For example, suppose you have GPS measurements of hurricanes every 10 minutes. Each GPS measurement records the hurricane name, location, time of recording, and the wind speed. You could create tracks of the hurricanes using the name of the hurricane as the track identification, and all hurricanes’ tracks would be generated. You could calculate statistics such as the mean, maximum, and minimum wind speed of each hurricane, as well as the count of measurements in each track.

In [21]:
from arcgis.geoanalytics.summarize_data import reconstruct_tracks

This example aggregate the numerous points into line segments showing the tracks followed by the hurricanes. The tool creates a feature layer item as an output which can be accessed once the processing is complete.

In [22]:
##usage example
result = reconstruct_tracks(years_50,
                            track_fields='Serial_Num',
                            method='GEODESIC')

In [23]:
result

<Item title:"Reconstruct_Tracks_1SRS6M" type:Feature Layer Collection owner:arcgis_python>

## Summarize attributes

The Summarize Attributes tool summarizes like field values to generate a summary table. The resulting layer displays the count of features that have been summarized, as well as any additional statistics that have been specified.

<center><img src="../../static/img/guide_img/ga/summarize_attributes.png" height="300" width="300"></center>

The [`summarize_attributes`]((https://developers.arcgis.com/rest/services-reference/enterprise/summarize-attributes.htm)) operation takes an input layer and summarizes and calculates statistics for like values. The most basic statistic is the count of features with a specified value, but other statistics can be calculated as well. You can also summarize values into time steps.

For example, if you have a point layer of store locations with a field representing the DISTRICT_MANAGER_NAME and you want to summarize coffee sales by manager, you can specify the DISTRICT_MANAGER_NAME field as the field to dissolve on, and all rows of data representing individual managers will be summarized. This means all store locations managed by Manager1 will be summarized into one row with summary statistics calculated. In this instance, statistics such as the number of stores and the sum of TOTAL_SALES for all stores that Manager1 manages will be calculated as well as for any other manager listed in the DISTRICT_MANAGER_NAME field.

In [24]:
from arcgis.geoanalytics.summarize_data import summarize_attributes

In [25]:
summarized_features = summarize_attributes(input_layer=years_50,
                                           fields='track_type')
summarized_features

<Item title:"Summarize_Attributes_Q74USW" type:Feature Layer Collection owner:arcgis_python>

## Summarize center and dispersion

The [`summarize_center_and_dispersion`](https://developers.arcgis.com/rest/services-reference/enterprise/summarize-center-and-dispersion.htm) task finds central features and directional distributions. It can be used to answer questions such as the following:

- Where is the center?

- Which feature is the most accessible from all other features?

- How dispersed, compact, or integrated are the features?

- Are there directional trends?

For an example, suppose you have used the GeoAnalytics tool Find Point Clusters to identify groups of power outages across an entire year. The result will be time enabled point representing cluster locations of power outages. However, you are interested in identifying the center of the power outages for visualization. To do this, you use Summarize Center And Dispersion a group by field of the outage cluster ids.

<center><img src="../../static/img/guide_img/ga/summarize_center_and_dispersion.png" height="300" width="300"></center>

Note: This tool is available in ArcGIS Enterprise version 10.9 and later.

In [6]:
from arcgis.geoanalytics.summarize_data import summarize_center_and_dispersion

In [ ]:
output_trends = summarize_center_and_dispersion(input_layer=years_50, 
                                                summary_type='MedianCenter', 
                                                output_name='directional trends')
output_trends

## Summarize within

The Summarize Within tool calculates statistics in areas where an input layer is within or overlaps a boundary layer. The area you are summarizing within can be an area layer or a hexagonal or square bin.

<center><img src="../../static/img/guide_img/ga/summarize_within.png" height="300" width="300"></center>

The [`summarize_within`]((https://developers.arcgis.com/rest/services-reference/enterprise/summarize-within.htm)) task finds features (and portions of features) that are within the boundaries of areas in the first input layer. The following are examples:

- Given a layer of watershed boundaries and a layer of land-use boundaries, calculate the total acreage of land-use type for each watershed.

- Given a layer of parcels in a county and a layer of city boundaries, summarize the average value of vacant parcels within each city boundary.

- Given a layer of counties and a layer of roads, summarize the total mileage of roads by road type within each county.

You can think of summarize_within as taking two layers and stacking them on top of each other. One of the layers, summary_polygons, must be a polygon layer, and imagine that these polygon boundaries are all colored red. The other layer, summarized_layer, can be any feature type—point, line, or polygon. After stacking these layers on top of each other, you peer down through the stack and count the number of features in summarized_layer that fall within the polygons with the red boundaries (summary_polygons). Not only can you count the number of features, you can calculate simple statistics about the attributes of the features in summarized_layer, such as sum, mean, minimum, maximum, and so on.

In [28]:
from arcgis.geoanalytics.summarize_data import summarize_within

In [30]:
##usage example
summarised_features = summarize_within(years_50, 
                                       bin_type="Square",
                                       bin_size=5,
                                       bin_size_unit='Miles',
                                       standard_summary_fields=[{"statisticType" : "average", "onStatisticField" : "Wind" }],
                                       output_name='summmrized_features')
summarised_features

<Item title:"summmrized_features" type:Feature Layer Collection owner:arcgis_python>